In [1]:
from player import *

In [18]:
x = 7
a = [z for z in range(x)]
# a = a - a[]
a, [z-a[x//2] for z in a]

([0, 1, 2, 3, 4, 5, 6], [-3, -2, -1, 0, 1, 2, 3])

In [12]:
((torch.rand(10, 12)<0.2).long() * torch.empty(10, 12).normal_(mean=0,std=1) * 0.7)

tensor([[ 0.0000,  0.6074, -0.0000, -0.0000,  0.0000,  0.0000, -0.0000,  0.0000,
          0.0000,  0.0000, -0.0000,  0.0000],
        [ 0.0000, -0.0000, -0.8116,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000,
          0.0000, -0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0472, -0.4051, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000,
          0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0478, -0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000, -0.0000,
         -0.1810, -1.1076,  0.0000,  0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, -0.6193,  0.0000, -0.0000, -0.0000,
         -0.7743, -1.1900, -0.0000, -0.0000],
        [-0.0000,  0.0000, -0.0000,  0.0000,  0.5668,  0.0000,  0.0000,  0.0000,
         -0.0000,  0.0000, -0.0000, -0.2586],
        [ 0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.1440,  0.3749,
         -0.0000,  0.6729, -0.0000,  0.0000],
        [ 0.6295, -0.0000, -0.4272,  0.0000,  0.0000,  0.7990, -0.0000, -0.0000,
          0.2842, -0.0000,  0.

In [2]:

pop_size = 5
num_trials = 5
move_limit = 5
num_gen = 5

mutation_rate = 0.1
mutation_change = 0.1
input_window_size = 8
input_size = input_window_size + 4 # input window size + food location

num_hidden_layers = 4
hidden_layers_size = [20] *  num_hidden_layers
output_size = 4

board_size, snake_count, food_count, set_seed = 10, 1, 1, False
# game = Game(board_size, snake_count, food_count)

player = geneticPlayer(pop_size, num_trials, mutation_rate, mutation_change, input_window_size, input_size,
                num_hidden_layers, hidden_layers_size, output_size, num_gen, move_limit)

# player.one_gen(board_size, snake_count, food_count, set_seed)
player.train_for_n_gen()

NameError: name 'board_size' is not defined

In [ ]:

for move in ['down', 'left', 'left', 'left', 'left', 'left', 'up', 
'up', 'up', 'right', 'right', 'right', 'right', 'right', 'down', 'down', 
'left', 'left', 'left', 'left', 'up', 'up', 'up', 'right', 'right', 'right', 
'right', 'right', 'down', 'down', 'down', 'down', 'down', 'left', 'left', 'left',
'up', 'up', 'up', 'up', 'up', 'left', 'left', 'left', 'left', 'down', 'down', 'down', 
'down', 'down', 'down', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 
'left', 'left', 'up', 'up', 'up', 'up', 'up', 'right', 'right', 'right', 'right', 'right', 'right', 
'right', 'right', 'right', 'down', 'down', 'down', 'down', 'down', 'left', 'left', 'left', 'left']:
    game.move(move)

In [ ]:

for move in ['down', 'left', 'left', 'left', 'left', 'left', 'up', 
'up', 'up', 'right', 'right', 'right', 'right', 'right', 'down', 'down', 
'left', 'left', 'left', 'left', 'up', 'up', 'up', 'right', 'right', 'right', 
'right', 'right', 'down', 'down', 'down', 'down', 'down', 'left', 'left', 'left',
'up', 'up', 'up', 'up', 'up', 'left', 'left', 'left', 'left', 'down', 'down', 'down', 
'down', 'down', 'down', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 
'left', 'left', 'up', 'up', 'up', 'up', 'up', 'right', 'right', 'right', 'right', 'right', 'right', 
'right', 'right', 'right', 'down', 'down', 'down', 'down', 'down', 'left', 'left', 'left', 'left']:
    game.move(move)